## Auditing and Exporting Detections

#### Before running this script, make sure that your Google Drive folder contains the orthomosaic GeoTiff (`step 0`) and no other GeoTIFF files, the `data.json` file (`step 1`), the `classes.csv` file (`step3`) and the `new_detections.json` file (`step 4`). If you want to add your new CNN detections to the manually annotated detections you created in VIA, also add the JSON file you exported using VIA (`step 2`). You will need to input that file name directly, since it is not standardized in our workflow.

<a href="https://colab.research.google.com/github/gl7176/GreySealCNN/blob/master/5_export_detections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
#####  <center> Be sure to update this hyperlink above if you clone and want to point to a different GitHub </center>

### Connect to our Google Drive folder and pull annotation CSV

In [2]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('data')
try:
  os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters

# set variable to the destination google drive folder you want to pull from
drive_folder = 'https://drive.google.com/drive/folders/1INuRNVKvKMy8L_Nb6lmoVbyvScWK0-0D'

# this bit points the code to that google drive folder
pointer = str("'" + drive_folder.split("/")[-1] + "'" + " in parents")

file_list = drive.ListFile(
    {'q': pointer}).GetList()

#    this bit pulls key files from the directory specified above
orthomosaic_file = {}
for f in file_list:
  # 3. Create & download by id.
  fname = os.path.join(local_download_path, f['title'])
  if fname.endswith(".tif") or fname.endswith(".json") or fname.endswith("classes.csv"):
    f_ = drive.CreateFile({'id': f['id']})
    f_.GetContentFile(fname)
    os.stat(fname)
    # if the file is a *.tif and larger than 100 mb we label it the orthomosaic
    if fname.endswith(".tif") and os.stat(fname).st_size > 10^8 :
      # if there are multiple orthomosaic files detected we spit an error
      if len(orthomosaic_file) != 0:
        raise Exception("more than one orthomosaic file identified based on size and type")
      orthomosaic_file = fname
      print("orthomosaic identified as " + orthomosaic_file)
print("all files pulled")

orthomosaic identified as data/Hay Island 2015.tif
all files pulled


### Set up the python environment and key variables

In [3]:
import os
import argparse
import numpy as np
import json
import csv
!pip install rasterio
import rasterio

!shapely.geometry import mapping, Polygon
!pip install fiona
import fiona # only required for exporting to shapefiles

     |████████████████████████████████| 18.3MB 246kB/s 
/bin/bash: shapely.geometry: command not found
     |████████████████████████████████| 14.8MB 3.9MB/s 


### Exporting to Shapefile

In [59]:
# open the output from our CNN
with open(local_download_path + '/new_detections.json') as f:
    detected_labels = json.load(f)

image_annotations = []
for key, value in detected_labels.items():
    annotation = [[key][0].split("/")[-1]]
    detections = []
    for item in value:
        box_w_label = []
        box = item['box']
        box_w_label.append(box)
        score = item['score']
        box_w_label.append(score)
        label = item['label']
        box_w_label.append(label)
        detections.append(box_w_label)
    annotation.append(detections)
    image_annotations.append(annotation)
for i in image_annotations:
   print(i)

['2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---93.png', [[[706, 23, 752, 55], 0.8100398778915405, 1], [[682, 41, 719, 77], 0.796779215335846, 1], [[677, 480, 726, 556], 0.7938922047615051, 0], [[918, 250, 959, 286], 0.7647950053215027, 1], [[753, 147, 794, 185], 0.7569835782051086, 1], [[532, 179, 582, 217], 0.7454524040222168, 1], [[480, 248, 526, 286], 0.7198755145072937, 1], [[593, 866, 680, 908], 0.7141700983047485, 0], [[839, 746, 894, 814], 0.7027513384819031, 0], [[425, 222, 465, 262], 0.6947418451309204, 1], [[328, 529, 407, 575], 0.6922022700309753, 0], [[928, 619, 971, 656], 0.6312804222106934, 1], [[645, 16, 682, 52], 0.6308165788650513, 1], [[465, 736, 520, 798], 0.6239029169082642, 0], [[901, 206, 941, 239], 0.5846998691558838, 1], [[768, 191, 812, 225], 0.5823211669921875, 1], [[764, 571, 807, 648], 0.5560154914855957, 0], [[816, 120, 853, 160], 0.5341529846191406, 1], [[864, 249, 894, 293], 0.5253627896308899, 1], [[618, 32, 657, 72], 0.5241472125053406, 1

In [56]:
# open the output from our CNN
with open(local_download_path + '/new_detections.json') as f:
    detected_labels = json.load(f)

image_annotations = []
for key, value in detected_labels.items():
    #print(str(key) + str(value))
    annotation = [key][0].split("/")[-1]
    detections = []
    for item in value:
        img_w_detect = [annotation, item]
        detections.append(img_w_detect)
    image_annotations.append(detections)
for i in image_annotations:
   print(i)

[['2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---93.png', {'box': [706, 23, 752, 55], 'label': 1, 'score': 0.8100398778915405}], ['2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---93.png', {'box': [682, 41, 719, 77], 'label': 1, 'score': 0.796779215335846}], ['2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---93.png', {'box': [677, 480, 726, 556], 'label': 0, 'score': 0.7938922047615051}], ['2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---93.png', {'box': [918, 250, 959, 286], 'label': 1, 'score': 0.7647950053215027}], ['2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---93.png', {'box': [753, 147, 794, 185], 'label': 1, 'score': 0.7569835782051086}], ['2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---93.png', {'box': [532, 179, 582, 217], 'label': 1, 'score': 0.7454524040222168}], ['2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---93.png', {'box': [480, 248, 526, 286], 'label': 1, 'score': 0.7198755145072937

In [45]:
print(detections[1][0][0])
print(len(detections[1][0][0]))
print(detections[1][0])
print(len(detections[1][0]))
print(detections[1])
print(len(detections[1]))

2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---145.png
67
['2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---145.png', [([...], {'box': [895, 244, 973, 293], 'label': 0, 'score': 0.8620665073394775}), ([...], {'box': [651, 11, 729, 55], 'label': 0, 'score': 0.7767825126647949})]]
2
(['2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---145.png', [([...], {'box': [895, 244, 973, 293], 'label': 0, 'score': 0.8620665073394775}), (...)]], {'box': [651, 11, 729, 55], 'label': 0, 'score': 0.7767825126647949})
2


In [ ]:
scores = []
for key, value in detected_labels.items():
    for item in value:
        score = item['score']
        scores.append(score)

In [ ]:
with open(local_download_path + '/data.json') as f:
  img_data = json.load(f)

image_bbox = []
for annotation in image_annotations:
    for detection in annotation[1]:
        try:
            image_box_list = []
            local_bounding_box = np.array([[detection[0][0], detection[0][1]], [detection[0][2], detection[0][1]], [detection[0][2], detection[0][3]], [detection[0][0], detection[0][3]]]).astype(int)
            image_located_bb = local_bounding_box + [img_data["image_locations"][annotation[0]]]
            image_box_list.append(image_located_bb)
            image_box_list.append(detection[1])
            image_box_list.append(detection[2])
            image_bbox.append(image_box_list)
            
        except ValueError: # if the image doesn't have a detection
            pass

In [ ]:
bbox = []
for annotation in image_bbox:
    total_box = []
    x1 = annotation[0][0][0]
    y1 = annotation[0][0][1]
    x2 = annotation[0][1][0]
    y2 = annotation[0][2][1]
    bounding_box = list([x1,y1,x2,y2])
    total_box.append(bounding_box)
    total_box.append(annotation[1])
    total_box.append(annotation[2])
    bbox.append(total_box)

In [ ]:
print(bbox[0])

[[8066, 4623, 8112, 4655], 0.8100398778915405, 1]


In [ ]:
# Malisiewicz et al.
# import the necessary packages
import numpy as np

def non_max_suppression(boxes, overlapThresh):
    # if there are no boxes, return an empty list
    if len(boxes) == 0:
        return []

    # if the bounding boxes are integers, convert them to floats -- this
    # is important since we'll be doing a bunch of divisions
    if boxes[0].dtype.kind == "i":
        boxes[0] = boxes[0].astype("float")

    # initialize the list of picked indexes
    pick = []

    # grab the coordinates of the bounding boxes
    x1 = []
    y1 = []
    x2 = []
    y2 = []

    for box_count in boxes[:,0]:
      x1.append(box_count[0])
      y1.append(box_count[1])
      x2.append(box_count[2])
      y2.append(box_count[3])
    
    x1 = np.array(x1)
    y1 = np.array(y1)
    x2 = np.array(x2)
    y2 = np.array(y2)

    # compute the area of the bounding boxes and grab the indexes to sort
    # (in the case that no probabilities are provided, simply sort on the
    # bottom-left y-coordinate)
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(boxes[:,1])

    # sort the indexes
    #idxs = np.argsort(idxs)

    # keep looping while some indexes still remain in the indexes list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the index value
        # to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        # find the largest (x, y) coordinates for the start of the bounding
        # box and the smallest (x, y) coordinates for the end of the bounding
        # box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        # compute the width and height of the bounding box
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        # compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]

        # delete all indexes from the index list that have overlap greater
        # than the provided overlap threshold
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > overlapThresh)[0])))

    # return only the bounding boxes that were picked
    return boxes[pick]#.astype("int")

In [ ]:
bboxes = np.array(bbox)
pick = non_max_suppression(bboxes, 0.6)
pick_list = pick.tolist()

In [ ]:
print(pick_list)

[[[8136, 11261, 8179, 11294], 0.9102174639701843, 1], [[8419, 10932, 8473, 11012], 0.887717604637146, 0], [[7392, 9367, 7463, 9431], 0.8841370940208435, 0], [[7863, 9912, 7932, 9979], 0.8687878251075745, 0], [[3714, 12101, 3754, 12136], 0.8631736636161804, 1], [[9175, 7604, 9253, 7653], 0.8620665073394775, 0], [[11276, 2059, 11318, 2094], 0.8537882566452026, 1], [[12840, 2365, 12916, 2415], 0.8289740681648254, 0], [[6873, 5117, 6915, 5151], 0.8218233585357666, 1], [[11996, 2700, 12041, 2736], 0.8149935603141785, 1], [[8066, 4623, 8112, 4655], 0.8100398778915405, 1], [[4638, 7455, 4700, 7514], 0.8005296587944031, 0], [[4369, 10745, 4420, 10813], 0.8000049591064453, 0], [[7389, 11260, 7473, 11316], 0.7982905507087708, 0], [[8042, 4641, 8079, 4677], 0.796779215335846, 1], [[8037, 5080, 8086, 5156], 0.7938922047615051, 0], [[8173, 11119, 8216, 11154], 0.7797330021858215, 1], [[8931, 7371, 9009, 7415], 0.7767825126647949, 0], [[12514, 2155, 12586, 2202], 0.7683605551719666, 0], [[7830, 9544

In [ ]:
# ingest back in the coordinates of detections within an image referenced by their filename
with open(local_download_path + '/data.json') as f:
    img_data = json.load(f)

# open the orthomosaic
dataset = rasterio.open(orthomosaic_file)

# pull labels from classes.csv
import csv
with open(local_download_path + '/classes.csv', "r") as f:
    reader = csv.reader(f, delimiter=",")
    labels_to_names = {int(i[1]):i[0] for i in reader}

geolocated_annotations_before_nms = []

for annotation in image_annotations:
    for detection in annotation[1]:
        try:            
            local_bounding_box = np.array([[detection[0][0], detection[0][1]], [detection[0][2], detection[0][1]], [detection[0][2], detection[0][3]], [detection[0][0], detection[0][3]]]).astype(int)
            image_located_bb = local_bounding_box + [img_data["image_locations"][annotation[0]]]
            
            geolocated_bb = []
            for point in image_located_bb:
                geolocated_bb.append(dataset.transform * point)
            geolocated_annotations_before_nms.append(geolocated_bb)
        except ValueError: # if the image doesn't have a detection
            pass

geolocated_annotations_after_nms = []
        
for box in pick_list:
    box_labeled = []
    image_located_bb = np.array([[box[0][0], box[0][1]], [box[0][2], box[0][1]], [box[0][2], box[0][3]], [box[0][0], box[0][3]]]).astype(int)
    geolocated_bb = []
    for point in image_located_bb:
        geolocated_bb.append(dataset.transform * point)
    box_labeled.append(geolocated_bb)
    box_labeled.append(labels_to_names[box[2]])
    geolocated_annotations_after_nms.append(box_labeled)

In [ ]:
print(geolocated_annotations_after_nms[0:5])

[[[(292130.18141, 5099948.34892), (292131.72640000004, 5099948.34892), (292131.72640000004, 5099947.16323), (292130.18141, 5099947.16323)], 1], [[(292140.3496, 5099960.1698900005), (292142.28982, 5099960.1698900005), (292142.28982, 5099957.29549), (292140.3496, 5099957.29549)], 0], [[(292103.44949, 5100016.40034), (292106.00052, 5100016.40034), (292106.00052, 5100014.10082), (292103.44949, 5100014.10082)], 0], [[(292120.37252000003, 5099996.81849), (292122.85169000004, 5099996.81849), (292122.85169000004, 5099994.41118), (292120.37252000003, 5099994.41118)], 0], [[(291971.29895, 5099918.16772), (291972.73615, 5099918.16772), (291972.73615, 5099916.91017), (291971.29895, 5099916.91017)], 1]]


In [ ]:
print("before NMS: " + str(len(geolocated_annotations_before_nms)))
print("after NMS: " + str(len(geolocated_annotations_after_nms)))

before NMS: 132
after NMS: 126


In [ ]:
# write out the detections as a shapefile

from collections import OrderedDict
import fiona
from fiona.crs import from_epsg

# Set output directory
output_dir = 'shapefile_output'

# create the dir if it doesn't already exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Define your schema as a polygon geom with a couple of fields
schema = {
    'geometry': 'Polygon',
    'properties': OrderedDict([
        ('ImageName', 'str'),
        ('Detection', 'str')
  ])
}

with fiona.open(output_dir + '/seal_detections.shp',
    'w',
    driver='ESRI Shapefile',
    crs=dataset.crs,
    schema=schema) as c:
    
    for num, polygon in enumerate(geolocated_annotations_after_nms):
      record = {
            'geometry': {'coordinates': [polygon[0]], 'type': 'Polygon'},
            'id': num,
            'properties': OrderedDict([('ImageName', orthomosaic_file),
                                       ('Detection', polygon[1])
                                       ]),
            'type': 'Feature'}
      c.write(record)

### Zip output folder for download

In [ ]:
# zip up the output directory into an archive for download
import subprocess
subprocess.call(['zip', '-r', '/content/' + output_dir + '.zip', '/content/' + output_dir])

from google.colab import files
files.download("/content/" + output_dir + ".zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### RetinaNet to Existing VIA

In [ ]:
# open the previous training data

with open(local_download_path + '/via_SealCNN_TrainingData.json') as f:
    existing_labels = json.load(f)

# add the new detections to the old via_region_data.json file

#"11_fiX1mEhK","[""2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---27.png""]",0,"[]","[2,509.275,929.174,89.376,48.904]","{}"

for filepath, detections in detected_labels.items():
    fn = filepath.split("/")[-1]
    # TODO is deep copy correct?
    for filename_size, metadata in existing_labels.items():
        if fn == metadata["filename"]:
            for detection in detections:
                # 'box' : [x1, y1, x2, y2]
                x1 = detection["box"][0]
                y1 = detection["box"][1]
                x2 = detection["box"][2]
                y2 = detection["box"][3]
                #print(x1,x2,y1,y2)
                metadata["regions"].append({'shape_attributes': {'name': 'rect', 'x': x1, 'y': y1, 'width': x2-x1, 'height': y2-y1}, 'region_attributes': {}})
    

In [ ]:
# write out new VIA file with additional detections

with open(local_download_path + '/via_region_data_detections.json', 'w') as fp:
    json.dump(existing_labels, fp)